# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/ll

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/deepseek-ai/DeepSeek-R1',
 '/hexgrad/Kokoro-82M',
 '/openbmb/MiniCPM-o-2_6',
 '/MiniMaxAI/MiniMax-Text-01',
 '/jinaai/ReaderLM-v2',
 '/models',
 '/spaces/hexgrad/Kokoro-TTS',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/HumanLLMs/Human-Like-DPO-Dataset',
 '/datasets/NovaSky-AI/Sky-T1_data_17k',
 '/datasets/DAMO-NLP-SG/multimodal_textbook',
 '/datasets/AI-MO/NuminaMath-CoT',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/dif

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community discussion page',
   'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://huggingface.co"))

In [22]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-R1\nUpdated\nabout 1 hour ago\n•\n5.57k\n•\n1.41k\nhexgrad/Kokoro-82M\nUpdated\nabout 1 hour ago\n•\n29.4k\n•\n2.18k\nopenbmb/MiniCPM-o-2_6\nUpdated\n31 minutes ago\n•\n24.1k\n•\n725\nMiniMaxAI/MiniMax-Text-01\nUpdated\n5 days ago\n•\n3.18k\n•\n450\njinaai/ReaderLM-v2\nUpdated\n7 minutes ago\n•\n13.1k\n•\n328\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n1.35k\n❤️\nKokoro TTS\nNow in 5 languages!\nRunning\non

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com'}, {'type': 'company page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Brochure

## **About Us**
### Hugging Face – The AI Community Building the Future
At Hugging Face, we believe in the power of collaboration within the machine learning community. Our platform serves as a hub for researchers, developers, and enthusiasts to come together and build innovative AI models, datasets, and applications. By creating an open-source environment, we empower our users to share their work and enhance their ML profiles.

## **Our Offerings**
- **Models:** Access over **400k+** cutting-edge machine learning models.
- **Datasets:** Browse through **100k+** diverse datasets for various use cases.
- **Spaces:** Host and collaborate on **150k+** applications to accelerate your AI projects.
  
### **Enterprise Solutions**
Hugging Face provides enterprise-grade services that include:
- Optimized Inference Endpoints
- Priority support and audit logs
- Secure access controls and dedicated support

Start at just **$20 per user/month** for enterprise features.

## **Customer Base**
Join over **50,000 organizations** utilizing Hugging Face, including:
- **Meta**
- **Amazon Web Services**
- **Google**
- **Microsoft**
- **Intel**
- And many more!

## **Our Open Source Commitment**
We are dedicated to fostering an open-source ecosystem, building essential ML tools alongside our community. Our popular libraries include:
- **Transformers** for state-of-the-art ML in PyTorch, TensorFlow, and JAX
- **Diffusers** for cutting-edge diffusion models
- **Tokenizers** for optimized text processing

## **Company Culture**
At Hugging Face, we thrive on innovation, collaboration, and community engagement. Our culture encourages:
- **Openness:** Sharing knowledge and resources to advance AI collectively
- **Support:** A nurturing environment where you can learn, grow, and contribute to meaningful projects
- **Diversity:** Embracing a range of perspectives to drive creativity and innovation

## **Careers at Hugging Face**
Join our passionate team at Hugging Face! We seek driven individuals who want to make an impact in the AI community. Explore our available positions and find your fit:
- Software Engineering
- Data Science
- Community Management
- Product Management

## **Connect with Us**
Stay updated and engage with our community through our various channels:
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://linkedin.com/company/hugging-face)
- [Discord](https://discord.gg/huggingface)

### **Join us in shaping the future of AI!**  
Sign up today at [Hugging Face](https://huggingface.co) and discover the power of community-driven AI development.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

## About Us
Hugging Face is at the forefront of the AI revolution, fostering a community that collaborates to build innovative machine learning (ML) models, datasets, and applications. Our platform empowers researchers, developers, and organizations to accelerate their AI projects while contributing to an inclusive, open-source ecosystem.

## Our Offerings

### Models
With over 400,000 models available, Hugging Face is the **home of machine learning**. From state-of-the-art natural language processing (NLP) to advanced computer vision, users can easily create, discover, and share models to enhance their projects.

### Datasets
Access a rich repository of over 100,000 datasets designed for various ML tasks, ensuring that users have the data needed to train their models efficiently.

### AI Spaces
Explore and run a variety of applications—150,000 strong and counting—where community members deploy creative AI solutions in practical scenarios. 

### Enterprise Solutions
Hugging Face provides **enterprise-grade** solutions for over 50,000 organizations, enhancing security and offering dedicated support to facilitate advanced AI development. 

## Our Community
At Hugging Face, we believe in the power of collaboration. Our platform hosts contributions from both indie creators and major enterprises such as **Meta**, **Amazon Web Services**, **Google**, and **Microsoft**. 

Join a vibrant community of both established organizations and innovative individuals committed to creating the future of AI. 

## Company Culture
At Hugging Face, we value:
- **Open Source**: We are dedicated to maintaining a free and inclusive development environment that allows all to contribute.
- **Collaboration**: Our platform is built on shared knowledge and teamwork, utilizing collective intelligence to push boundaries.
- **Innovation**: We encourage creative thinking and experimentation, enabling our team and community members to stay ahead in the field of AI.

## Careers at Hugging Face
We are always looking for passionate individuals to join our team! If you have a drive for innovation and a desire to contribute to the world of artificial intelligence, explore our careers page to discover opportunities across various departments, including engineering, product management, research, and community engagement.

## Why Choose Hugging Face?
- Leverage **cutting-edge technology** with our open-source tools like Transformers, Diffusers, and Accelerate.
- Gain insights and support from a **robust community** of AI enthusiasts and professionals.
- Benefit from our **scalable solutions** designed for both individuals and enterprises.
  
Together, we are building the future of AI. Join us at Hugging Face!

---

> For more information, visit our website: [Hugging Face](https://huggingface.co)  
> Follow us on social media: [Twitter](https://twitter.com/huggingface) | [LinkedIn](https://www.linkedin.com/company/hugging-face) | [Discord](https://discord.com/invite/huggingface)  
> Explore career opportunities at: [Jobs](https://huggingface.co/jobs)

In [23]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# 🎉 Welcome to the Hugging Face Brochure! 🎉

### 🌟 Hugging Face - The AI Community That's Practically Cuddly! 🌟

Have you ever wanted to work in a place where the algorithms are as friendly as the employees? Well, snuggle up to **Hugging Face**, where we build the future one model at a time! Ready to explore? Let's dive in!

---

### 🤖 What Do We Do? 

At Hugging Face, we provide the **ultimate playground** for all things machine learning (ML). Collaborate on:
- **400,000+ Models**: Like a buffet, but instead of food, it’s algorithms!
- **100,000+ Datasets**: Because data is the new oil, and we’re drilling deep!
- **150,000+ Applications**: Our apps are so versatile, they put a Swiss Army knife to shame.

And we do it **open source** because sharing is caring, and we’re also kind of like digital Santa Claus! 🎅

---

### 🏢 Company Culture - No Robots Allowed!

Our company culture is as warm as a cozy blanket, with:
- **Inclusivity**: Everyone is welcome - even your grandma (in spirit!).
- **Fun Fridays**: Because who doesn’t love a good meme battle?
- **Coffee on Tap**: Fuel for your inner nerd & the key to AI success!

And did we mention? We like to refer to our break room as the **'Hug Zone.'** 🤗

---

### 🌐 Customers - Big Names, Bigger Hugs!

With more than **50,000 organizations** using Hugging Face, our customer list reads like a tech startup dream:
- **Microsoft**
- **Amazon**
- **Google**
- **Grammarly**: They say they help with the writing; we think they just come for the hugs!

Join the cool kids in the AI playground. No need for a password, just a cute smile and a genuine interest! 😏

---

### 💼 Careers - You're Invited to Our Hug Fest!

Looking for a job? Want to be part of a community where your coding can directly impact *the future*? Check out our careers. Perks include:
- 🤗 Working with folks who would literally hug a transformer (and we don’t mean the toy!).
- 🌈 A diverse team that believes in *open source* - you get to share your own magic while learning from others!
- 💻 Opportunities to innovate and collaborate: your wildest ML dreams can come true here!

Openings are often available, but please—no velvet ropes. We’re all about accessibility! 👐

---

### 🌈 Ready to Embrace the Future? 

Whether you’re a customer, an investor, or a potential recruit looking for a fun place to call home, **Hugging Face** is waiting for you with tender algorithms and open arms. So, what are you waiting for? Let's get this AI party started! 🚀

---

**Follow Us for More Adventures:**
- [Hugging Face](https://huggingface.co)
- [Twitter](https://twitter.com/huggingface)
- [GitHub](https://github.com/huggingface)

**Let's Hug It Out!**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>